In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
import cv2

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 8
%config InlineBackend.figure_format = 'retina'

In [2]:
from keras.applications import VGG16
from keras.models import Sequential, load_model
from keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.metrics import Recall

vgg = VGG16(include_top=False, weights="imagenet")
vgg.trainable = False

dense = load_model("modele-VGG16") # ou 'modele-VGG16.h5'
modele = Sequential([
    Input((227, 227, 3)),
    vgg,
    dense
], name="complet")
modele.trainable = False

dense.summary()
recall = Recall(name="recall")
modele.compile('adam', 'categorical_crossentropy', metrics=[recall])

Model: "dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3211392   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 3,219,973
Trainable params: 0
Non-trainable params: 3,219,973
_________________________________________________________________


In [3]:
from os.path import join
NOM = "BowlPlace1Subject1"
VIDEO = join("VIDEOS", NOM + ".mp4")
BOITE = join("GT", NOM + "_2_bboxes.txt")

## Lecture des boites fournies
Lit le fichier contenant les informations de reconnaissance d'objets pour avoir un point de départ.

In [4]:
from os.path import exists, join
from typing import List, Tuple
import csv

def lecture_information_boite_englobante(nom: str) -> List[List[Tuple[int, int, int, int]]]:
    """ Lit les boites englobante fournies pour l'exemple. """
    resultats = list()
    
    with open(join("GT", nom + "_2_bboxes.txt"), "r") as fichier:
        lecteur = csv.reader(fichier, delimiter=" ")

        for ligne in lecteur:
            frame_nb = int(ligne[0])
            boite_nb = int(ligne[1])
            boites = list()
            for i in range(boite_nb):
                boite = (int(ligne[i * 4 + 2]), int(ligne[i * 4 + 3]), int(ligne[i * 4 + 4]), int(ligne[i * 4 + 5]))
                boites.append(boite)
            resultats.append(boites)
    
    return resultats

boites_englobantes = lecture_information_boite_englobante(NOM)

## Recherche de l'objet sur la prochaine image
À partir de la zone désignée dans l'image précédante, tente de retrouver l'objet dans la nouvelle image.

La fonction `evaluation_boite` prend en entrée une liste de boite (x, y, hauteur, largeur) et retourne l’évalution de chacune d’elle par le réseau de neurones.
L’évalution se fait par groupes de boites pour aller plus vite.

La fonction `tracking_move_box` tente de repositionner la boite en explorant les alentours et en raffinant les déplacements.
La longueur du _pas_ de déplacement diminue à chaque itération et l’exploration recommence à proximité de la meilleure position connue.
Cet algorithme recherche une maximum local avant de raffiner sa position.

Il n'est pas performant quand le maximum atteind la valeur 1.
En effet le réseau est entraîné à reconnaitre les objets même incomplet et peut obtenir une grande certitude sans avoir l'image dans son intégralité.

In [5]:
from PIL import Image

def evaluation_boite(image: Image, boites: list) -> np.array:
    """ Évalue les boites englobantes.
    Passer plusieurs boites englobantes en même temps permet d'accélérer le calcul.
    """
    #print(f"evaluation de {len(boites)} boites")
    extraits = list(np.asarray(image\
        .resize((227, 227), box=((boite[0], boite[1], boite[0] + boite[2], boite[1] + boite[3]))))
        for boite in boites)
    return modele.predict(np.array(extraits))

def tracking_move_box(image: Image.Image, boite_pre: (int, int, int, int), categorie: int) -> (int, int, int, int):
    """Calcul le mouvement de la boite.
    @param image: image courante
    @param boite_pre: ancienne boite englobante
    @return la nouvelle position de la boite englobante
    """
    assert(isinstance(boite_pre, tuple)), type(boite_pre)
    assert isinstance(boite_pre[0], int) and isinstance(boite_pre[1], int) and isinstance(boite_pre[2], int)
    
    valeurs = evaluation_boite(image, (boite_pre,))
    evaluations = {boite_pre: valeurs[0][categorie]}  # categorie
    meilleur_pos = boite_pre
    #print(meilleur_pos, valeur[categorie])
    #print(boite_cible, valeurs[1][categorie])

    # Modifications légères de la boite en maximisant la correspondance à la catégorie.
    # la déplacer un peu et réessayer
    # Méthode diamand sur 4D (x, y, w, h) ou (x, y, ratio, zoom)
    pas = 32
    while pas >= 8:
        # Liste des positions évaluées mais pas leurs voisins
        exploration = [meilleur_pos]
        # Liste des positions évaluées et aussi leurs voisins
        explores = set()

        while len(exploration) > 0 and (len(explores) < 4 or \
            max(explores, key=evaluations.__getitem__) < max(exploration, key=evaluations.__getitem__)):
            # Une position plus favorable nécessite d'étudier ses voisins
            pos = max(exploration, key=evaluations.__getitem__)
            exploration.remove(pos)

            # propositions de nouvelles boites englobantes
            propositions = list(boite_v
                for boite_v in boites_voisines(pos, image.size, pas=pas)
                    if boite_v not in evaluations.keys())
            propositions = list(boite_v for boite_v in propositions if boite_v not in evaluations)

            # évalue toutes les boites proposées
            valeurs = evaluation_boite(image, propositions)
            for i, boite_v in enumerate(propositions):
                # note pour la catégorie considérée
                evaluations[boite_v] = valeurs[i][categorie]

            exploration.extend(propositions)
            explores.add(pos)

        meilleur_pos = max(explores, key=evaluations.__getitem__)
        print(f"\tpas={pas}, pos={meilleur_pos}, valeur={evaluations[meilleur_pos]}")
        pas = int(pas / 2)
    return meilleur_pos

Cette version du tracking est plus légère et ne réalise qu’une seule exploration des environs.
Toutes les boites candidates sont évaluées d'un seul coup.

In [6]:
def tracking_leger(image: Image.Image, boite_pre: (int, int, int, int), categorie: int) -> (int, int, int, int):
    pas = 2
    demi_taille = 5
    positions = list()
    evaluations = {}
    positions.append(boite_pre)
    for dx in range(-demi_taille, demi_taille, pas):
        for dy in range(-demi_taille, demi_taille, pas):
            positions.append((boite_pre[0] + dx, boite_pre[1] + dy, boite_pre[2], boite_pre[3]))
    valeurs = evaluation_boite(image, positions)
    for i, boite_v in enumerate(positions):
        evaluations[boite_v] = valeurs[i][categorie]
    return max(positions, key=evaluations.__getitem__)


## Tracking sur vidéo
Cette boucle doit retrouver la position de la boite englobante en fonction de sa dernière position.

Utiliser n'importe quelle touche pour fermer la fenêtre.

In [ ]:
from utils.evaluateBBox import *

assert exists(VIDEO), "le fichier vidéo n'existe pas"
PAS_FRAME = 5 # ignore une partie des frames
DEPART = 0

frame = 0
video = cv2.VideoCapture(VIDEO)
tracking = False
categorie = 0
sequence = list()

while video.isOpened():
    # print(f"* frame {frame}")
    ret, f = video.read()
    if ret == False:
        print("erreur")
        break

    f = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(f)
        
    # découper une boite
    if not tracking and len(boites_englobantes[frame]) > 0:
        tracking = True
        sequence.append(boites_englobantes[frame][0])
        valeurs = evaluation_boite(image, (sequence[-1],))
        categorie = np.argmax(valeurs[0])
        DEPART = frame % PAS_FRAME
        print("tracking pour la catégorie", categorie)
        
    elif tracking and frame % PAS_FRAME == DEPART:
        boite_pre = sequence[-PAS_FRAME]
        boite_cible = boites_englobantes[frame][0]
        print(f"boite précédante: {boite_pre}, boite cible: {boite_cible}")
        
        meilleur_pos = tracking_move_box(image, boite_pre, categorie)
        # meilleur_pos = tracking_leger(image, boite_pre, categorie)

        print(f"ajustement de {boite_pre} à {meilleur_pos}")
        print(f"score: {IoU(boite_cible, meilleur_pos)}")
        sequence.append(meilleur_pos)
        x, y, w, h = meilleur_pos

        ff = f.copy()
        cv2.rectangle(ff, (x, y), (x+w, y+h), (0, 255, 0), 2) 
        cv2.imshow("frame "+str(frame), ff)
        cv2.waitKey(30000)
        cv2.destroyAllWindows()
        
    else:
        sequence.append(None)
    frame += 1

print("out")
video.release()